
<div class="text-center">
<h1 class="display-1" >Data Mining </h1>
<h3> Homework 2</h3>
<h3> Di Santo Nicola - 1853049</h3>
From:  
https://github.com/nicoDs96/Document-Similarity-using-Python-and-PySpark/blob/master/LSH/DM_HW2_Ex2.ipynb
</div>

# Exercise 2

## Libraries

In [1]:
import pandas as pd
import re, hashlib, math, time
from random import randint, seed
seed(1631996)


## HashFamily
The Hash Family is composed by always the same function but different member of the family are given by different salt values (i parameter of the class constructor). The only callable method is <i>get_hash_value</i> that will return the hash value of the element passed as parameter as an integer. 

In [2]:
class hashFamily:
    def __init__(self, i):
        self.resultSize = 8 # how many bytes we want back
        self.maxLen = 20 # how long can our i be (in decimal)
        self.salt = str(i).zfill(self.maxLen)[-self.maxLen:]
        
    def get_hash_value(self, el_to_hash):
        return int(hashlib.sha1(str(el_to_hash).encode('utf-8') + self.salt.encode('utf-8')).hexdigest()[-self.resultSize:], 16)
    

## Shingling
<p>The clas will mantain the k parameter as attribute. Once an object of this class is instaciated it can produce either normal k-shingles or hashed k-shingles of any document passed as parameter. To obtain hashed shingles we must first compute normal shingles and then invoke the method <i>get_hashed_shingles</i> that takes as input a set of shingles instead of a document.</p> 
<p>An example of usage is: <br>
<code>doc = "Ciao questo é uno documento!                     xd 1111@\n\n\n\n\n\n\n\n\n\n\n\n\n\n"
shingler_instance = shingler()
shingles_set = shingler_instance.get_shingles(doc)
hashed_shingles = shingler_instance.get_hashed_shingles(shingles_set ) </code>

or in a more compact way:<br>
<code>doc = "Ciao questo é uno documento!                     xd 1111@\n\n\n\n\n\n\n\n\n\n\n\n\n\n"
shingler_instance = shingler()
hashed_shingles = shingler_instance.get_hashed_shingles(shingler_instance.get_shingles(doc) ) 
</code> 
or:<br>
<code>doc = "Ciao questo é uno documento!                     xd 1111@\n\n\n\n\n\n\n\n\n\n\n\n\n\n"
hashed_shingles = shingler().get_hashed_shingles(shingler_instance.get_shingles(doc) ) 
</code> 
</p>

<p>Before shingling a string some simple normalization is performed: multiple spaces and muliple newlines character are replaced with just one space.</p>

In [3]:
class shingler:
    def __init__(self, k):
        
        if k > 0:
            self.k = int(k)
        else:
            self.k = 10
        
    #inner class utility
    def process_doc(self, document):
        return re.sub("( )+|(\n)+"," ",document).lower()
    
    def get_shingles(self, document):
        shingles = set()
        document= self.process_doc(document)
        for i in range(0, len(document)-self.k+1 ):
            shingles.add(document[i:i+self.k])
        return shingles
    
    def get_k(self):
        return self.k
    
    #return sorted hash
    def get_hashed_shingles(self, shingles_set):
        hash_function = hashFamily(0)
        return sorted( {hash_function.get_hash_value(s) for s in shingles_set} )
        

## Minhash
<p>So here’s how to compute the MinHash signature for a given document. 
<ol>
    <li>Generate n random hash functions $h_1,h_2,...,h_n$.</li> 
    <li>Take the first hash function, and apply it to all of the shingle values in a document. Find the minimum hash value produced and use it as the first component of the MinHash signature.</li>
    <li> Now take the second hash function, and again find the minimum resulting hash value, and use this as the second component. And so on...</li>

So if we have 10 random hash functions, we’ll get a MinHash signature with 10 values for each set.
We’ll use the same 10 hash functions for every document in the dataset and generate their signatures as well. Then we can compare the documents by counting the number of signature components in which they match.</p>
<p>The class implements a method to sign a single set of shinglings (either hashed or not) and then another method that iteratively applies the first method to all the set to produce the signature matrix.</p>
<p>Example of usage:<br>
    <code>doc = "Ciao questo é una stringa"
doc2 = "Ciao questo é un altro documento"
shingler_inst = shingler()        
shinglings = shingler_inst.get_shingles(doc) 
shinglings2 = shingler_inst.get_shingles(doc2)
minhash_sig = minhashSigner().compute_signature_matrix([shinglings,shinglings2])
</code></p>

In [4]:
 
class minhashSigner:
    def __init__(self, sig_size):
        self.sig_size=sig_size
        self.hash_functions = [hashFamily(randint(0,10000000000)) for i in range(0,sig_size)]
    
    def compute_set_signature(self, set_):
        set_sig = []
        for h_funct in self.hash_functions:
            min_hash = math.inf
            for el in set_:
                h = h_funct.get_hash_value(el)
                if h < min_hash:
                    min_hash = h
                
            set_sig.append(min_hash)
        
        return set_sig
    
    #return a list of lists that can be seen as the signature matrix
    def compute_signature_matrix(self, set_list):
        signatures = []
        for s in set_list:
            signatures.append( self.compute_set_signature(s) )
            
        return signatures
    

   ## LSH (Locality Sensitive Hashing)
   <p>The class performs the following tasks:<ol>
    <li>Divide signature matrix into bands with <code>get_signature_matrix_bands(sig_matrix, bands_nr, sign_len)</code>. The method will return a list of string where each string represent a column of the signature matrix for the given band. It is done to hash the entire column next, when producing buckets. </li>
    <li>Compute buckets for each band with <code>get_band_buckets(band, hash_funct)</code>, that will take as input a band b and an hash function h and will return a dict containing as key the bucket ids and as value a list of document ids that hashed to that bucket for the given band b: $\{bucket_{id}: [doc_i, doc_k,...]\}$ .</li>
    <li>With <code>get_candidates_list(buckets)</code> we are going to take as input a list of buckets (whose union is the signature matrix) and will produce as output a set of tuples: those tuples represent documents that hashed in the same bucket.</li>
    <li>With <code>check_candidates(candidates_list, threshold, sigs)</code> we take all the candidates from all the bands and check if effectively their signatures produce a match with approximate jaccard similarity greater than threshold we give as parameter.</li>
<li>With <code>get_similar_items(sig_matrix, bands_nr, sign_len)</code> we put toghether all the methods listed above and return the ids of similar documents that respect threshold value.</li></ol>
</p><p>The similarity threshold is passed as value to the constructor of the class, default is $0.8$.</p>
<p>Example of usage:<br>
<code>shingler_inst = shingler(10)
doc = "Ciao questa é una stringa NMVKJHFVSAh skjdg;sabdks"
shinglings = shingler_inst.get_shingles(doc) 
shinglings = shingler_inst.get_hashed_shingles(shinglings)

doc2 = "asbdkjasjcbsdakvfk.sdjB  Ciao questo a una stringa un po' diversa aaaaaaaa"
shingle2 = shingler_inst.get_shingles(doc2)
shingle2 = shingler_inst.get_hashed_shingles(shingle2)

doc3 = "bbbbbbbbbbbbbaaaaaaa Ciao questo é una stringa un po' diversa "
shingle3 = shingler_inst.get_shingles(doc3)
shingle3 = shingler_inst.get_hashed_shingles(shingle3)

signer = minhashSigner(100)
signature_matrix = signer.compute_signature_matrix([shinglings,shingle2,shingle3])

lsh_instance = lsh()
bands = lsh_instance.get_signature_matrix_bands(signature_matrix,10,100) # bands b=10, signature length n=100
similar_docs = set()
for band_id, elements in bands.items():
    buckets = lsh_instance.get_band_buckets(elements, hash_funct=hashFamily(1))
    candidates = lsh_instance.get_candidates_list(buckets)
    for sim_tuple in lsh_instance.check_candidates(candidates, 0.8, minhash_sigs):
        similar_docs.add( sim_tuple)    
print(similar_docs)
</code>
or compact:<br>
<code>signer = minhashSigner(100)
signature_matrix = signer.compute_signature_matrix([shinglings,shingle2,shingle3])
lsh_instance.get_similar_items(minhash_sigs,10,100)</code></p>

In [5]:
class lsh:
    def __init__(self, threshold=0.8):
        self.threshold = threshold
        
    def get_signature_matrix_bands(self, sig_matrix, bands_nr, sign_len): 
        #bands_nr = b
        #sign_len = n
        r = int(sign_len/bands_nr) #number of rows in each band
        bands = {} # {band_nr: [col_1,col_2,...]} where col_1 is all the values of Sig(S_i) for band b.
        for i in range(0,bands_nr):
            bands[i] = []
        
        # put Subsets of the columns of signature matrix into the appropriate bucket and cosider a column 
        # as a unique block so that we can hash the entire column.
        # Basically a band is a list of element, where each element is a subset of a signature of a given set.
        for signature in sig_matrix: 
            
            for i in range(0, bands_nr):
                idx = i*r    
                bands[i].append(' '.join(str(x) for x in signature[idx:idx+r]) ) 
                    
        return bands

    #band is a list 
    # construct a dictionary {hash(band_column): doc_id that produced this hash}
    def get_band_buckets(self, band, hash_funct):
        buckets = {}
        for doc_id in range(0,len(band)):
            value = hash_funct.get_hash_value( band[doc_id] )
            if value not in buckets:
                buckets[value] = [doc_id]
            else:
                 buckets[value].append(doc_id)
                
        return buckets
    
    def get_candidates_list(self, buckets):
        candidates = set()
        # buckets is a dictionary containing key=bucket, value= list of doc_ids that hashed to bucket
        for bucket,candidate_list in buckets.items():
            if len(candidate_list) > 1:
                for i in range(0,len(candidate_list)-1):
                    for j in range(i+1,len(candidate_list)):  
                        pair = tuple(sorted( (candidate_list[i],candidate_list[j]) ))
                        candidates.add(pair)
                
        return candidates #ie a set of couples, each couple is a candidate pair
    
    def check_candidates(self, candidates_list, threshold, sigs):
        similar_docs = set() #set of tuples
        # similar_pair is a couple containing doc_ids of documents that hashed to same bucket
        for  similar_pair in candidates_list:
            #for all the pairs of document in the list check similarity of their signatures
            doc_id_1 = similar_pair[0]
            doc_id_2 = similar_pair[1]
            signature_1 = set(sigs[doc_id_1]) #get the i-th column from signature matrix where i is doc_id in the collision list
            signature_2 = set(sigs[doc_id_2])
            js = len(signature_1.intersection(signature_2)) /len(signature_1.union(signature_2))
            
            if js >= threshold:
                similar_docs.add( tuple(sorted((doc_id_1,doc_id_2) )) )
                        
                        
        return similar_docs
    
    def get_similar_items(self, sig_matrix, bands_nr, sign_len):
        similar_docs = set()
        #divide signature matrix into bands
        bands = lsh_instance.get_signature_matrix_bands(sig_matrix,bands_nr,sign_len)
        
        #for all the bands
        for band_id, elements in bands.items():
            #produce the buckets for the given band (band_id) with a random hash function
            buckets = lsh_instance.get_band_buckets(elements, hash_funct=hashFamily(randint(0,10000000000)))
            #Get all the candidate pairs
            candidates = lsh_instance.get_candidates_list(buckets)
            #Check all candidate pairs' signatures
            for sim_tuple in lsh_instance.check_candidates(candidates, self.threshold, sig_matrix):
                similar_docs.add( sim_tuple)

        return similar_docs #return all the similar signatures that respect the threshold
                
        

## Bruteforce Shingles Comparison

In [6]:
class bfsc():
    def compare_shingles_set_js(self, set1, set2):
        return len(set1.intersection(set2))/len(set1.union(set2))

## Test on Kijiji Dataset

In [8]:
print("Loading dataset...")
dataset=pd.read_csv("../data/dataset_rent_rome_kijiji.tsv", sep="\t")
dataset['doc_id']=dataset.index
doc_nr = dataset['doc_id'].max()
print("Dataset loaded correctly.")
print("Producing Shingles...")
start_time = time.time()
#an array where the index i represent the document_id and the element shingling_list[i] the hashed shingles for document document_id
shingling_list = [None] * (doc_nr +1) 
shingling_size = 10
signature_size = 50
bands_nr = 10

shingler_inst = shingler(shingling_size)
signer = minhashSigner(signature_size)


#produce hashed shinglings for all documents
for index, row in dataset.iterrows():
    doc = row['Title']+" "+row['Short Description']
    i = row['doc_id']
    
    shinglings = shingler_inst.get_hashed_shingles( shingler_inst.get_shingles(doc) )
    shingling_list[i] = shinglings

end_time = time.time()
print("Shingles produced in:\t %.2f seconds."%(end_time - start_time))


start_time = time.time()
print("Computing signature matrix...")
#produce a signature for each shingle set
signature_matrix = signer.compute_signature_matrix( shingling_list )
end_time = time.time()
print("Signature Matrix computed in:\t %.2f seconds." %(end_time - start_time))





Loading dataset...
Dataset loaded correctly.
Producing Shingles...
Shingles produced in:	 1.57 seconds.
Computing signature matrix...
Signature Matrix computed in:	 43.25 seconds.


In [9]:
lsh_instance = lsh(threshold=0.8)
start_time = time.time()
print("Computing LSH similarity...")
lsh_similar_itemset = lsh_instance.get_similar_items(signature_matrix, bands_nr, signature_size)
end_time = time.time()
lsh_computation_time = end_time - start_time
print("LSH Similarity computed in:\t %.2f seconds.\nSimilar Elements Found: %d" %(lsh_computation_time,len(lsh_similar_itemset)))


Computing LSH similarity...
LSH Similarity computed in:	 0.73 seconds.
Similar Elements Found: 10296


### Bruteforce Comparison

In [10]:
#computing only for elements into similarity set 
brute_force_comparer = bfsc()
brute_force_similar_items = set()
start_time = time.time()
false_positive = 0

#for couple in lsh_similar_itemset:
#    i = couple[0]
#    j = couple[1]
#    similarity = brute_force_comparer.compare_shingles_set_js(set(shingling_list[i]),set(shingling_list[j]))
#    if similarity >= 0.8:
#        brute_force_similar_items.add( (i,j) )
#    else:
        #print("Document %d and %d have only %f of similarity."%(i,j,similarity))
#        false_positive = false_positive+1

#print("Brute Force Similarity Tooks:\t%f seconds\nDetected %d False Positives on %d\nSimilar docs are:\t%s"%(bf_computation_time, false_positive,len(lsh_similar_itemset),brute_force_similar_items))


for i in range(0,doc_nr-1):
    for j in range(i+1,doc_nr):
        similarity = brute_force_comparer.compare_shingles_set_js(set(shingling_list[i]),set(shingling_list[j]))
        if similarity >= 0.8:
            brute_force_similar_items.add( (i,j) ) 
            
end_time = time.time()
bf_computation_time = end_time - start_time      


#print("Brute Force Similarity Tooks:\t%.2f seconds\nSimilar docs are:\t%d"%(bf_computation_time, brute_force_similar_items))


## Example 1
Similar document with bruteforce search

In [11]:
for i in range(0,3):
    docs = brute_force_similar_items.pop()
    print("Doc 1:")
    print(dataset.iloc[docs[0]] )
    print("Is similar to\nDoc2:")
    print(dataset.iloc[docs[1]] )


Doc 1:
Title                                     Centro storico appartamento 
Short Description    In Via XX Settembre in stabile d''epoca con se...
Location                                                         Roma 
Price (Euro)                                                    3.670 
Timestamp                                              2 marzo, 02:05 
Url Adv              https://www.kijiji.it/annunci/affitto/roma-ann...
doc_id                                                            2272
Name: 2272, dtype: object
Is similar to
Doc2:
Title                                     Centro storico appartamento 
Short Description    In Via XX Settembre in stabile d''epoca con se...
Location                                                         Roma 
Price (Euro)                                                    3.670 
Timestamp                                          17 febbraio, 02:00 
Url Adv              https://www.kijiji.it/annunci/affitto/roma-ann...
doc_id                  

## Example 2
Similar items with LSH

In [12]:
for i in range(0,3):
    docs = lsh_similar_itemset.pop()
    print("Doc 1:")
    print(dataset.iloc[docs[0]] )
    print("Is similar to\nDoc2:")
    print(dataset.iloc[docs[1]] )

Doc 1:
Title                                     Centro storico appartamento 
Short Description    In Via XX Settembre in stabile d''epoca con se...
Location                                                         Roma 
Price (Euro)                                                    3.670 
Timestamp                                              2 marzo, 02:05 
Url Adv              https://www.kijiji.it/annunci/affitto/roma-ann...
doc_id                                                            2272
Name: 2272, dtype: object
Is similar to
Doc2:
Title                                     Centro storico appartamento 
Short Description    In Via XX Settembre in stabile d''epoca con se...
Location                                                         Roma 
Price (Euro)                                                    3.670 
Timestamp                                          17 febbraio, 02:00 
Url Adv              https://www.kijiji.it/annunci/affitto/roma-ann...
doc_id                  

## Execution Report

In [13]:
report = '''LSH\n%.2f seconds to execute\n%d similar documents found\n\nBruteforce search\n%.2f seconds to execute\n%d similar documents found.\n\nThey find %d common similarities.
'''

print(report %(lsh_computation_time, len(lsh_similar_itemset),bf_computation_time,len(brute_force_similar_items),len(brute_force_similar_items.intersection(lsh_similar_itemset)) ))

LSH
0.73 seconds to execute
10293 similar documents found

Bruteforce search
213.02 seconds to execute
10355 similar documents found.

They find 10291 common similarities.



## Ant additional cells from orginal notebook

In [14]:
dataset.head()

,Title,Short Description,Location,Price (Euro),Timestamp,Url Adv,doc_id
0,Studio accessoriato vicino metro A Furio Camillo,Affitto studio a professionisti preferibilment...,Roma,450,"12 ottobre, 11:32",https://www.kijiji.it/annunci/affitto/roma-ann...,0
1,"Negozio 169Mq per laboratorio, ufficio, studio...","Privato affitta negozio 169 mq, al piano terra...",Prenestino / Casilino,1.700,"12 ottobre, 08:45",https://www.kijiji.it/annunci/affitto/roma-ann...,1
2,Negozio in tiburtina centro,Negozio c/1 roma tiburtina centro via eugenio ...,Tiburtino / Collatino,6.000,"17 October, 21:20",https://www.kijiji.it/annunci/affitto/roma-ann...,2
3,Studio medico via anapo parco nemorense,"Studio medico avviato, composto da tre studi c...",Trieste / Somalia / Salario,200,"17 October, 20:22",https://www.kijiji.it/annunci/affitto/roma-ann...,3
4,Cerco: Appartamento per donna lavoratrice refe...,"Donna lavoratrice, non residente, con reddito ...",Roma,Contatta l'utente,"17 October, 19:39",https://www.kijiji.it/annunci/affitto/roma-ann...,4


In [15]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2627 entries, 0 to 2626
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Title              2627 non-null   object
 1   Short Description  2627 non-null   object
 2   Location           2627 non-null   object
 3   Price (Euro)       2627 non-null   object
 4   Timestamp          2627 non-null   object
 5   Url Adv            2627 non-null   object
 6   doc_id             2627 non-null   int64 
dtypes: int64(1), object(6)
memory usage: 143.8+ KB


In [18]:
lsh_similar_itemset.pop()

(485, 1066)

In [19]:
dataset.iloc[485]

Title                 Negozi C1 in via Caianello, Roma.38/43/50/67Mq. 
Short Description    Privato affitta negozi vari tagli in via Caian...
Location                                        Prenestino / Casilino 
Price (Euro)                                                      650 
Timestamp                                         23 settembre, 06:35 
Url Adv              https://www.kijiji.it/annunci/affitto/roma-ann...
doc_id                                                             485
Name: 485, dtype: object

In [20]:
dataset.iloc[1066]

Title                 Negozi C1 in via Caianello, Roma.38/43/50/67Mq. 
Short Description    Privato affitta negozi vari tagli in via Caian...
Location                                        Prenestino / Casilino 
Price (Euro)                                                      650 
Timestamp                                         23 settembre, 06:35 
Url Adv              https://www.kijiji.it/annunci/affitto/roma-ann...
doc_id                                                            1066
Name: 1066, dtype: object

In [33]:
# find all the matching doc_id for a given doc_id
def find_matching_doc(doc_id):
    return [couple for couple in lsh_similar_itemset if doc_id in couple]


def get_matching_doc_set(doc_id):
    sim_docs = find_matching_doc(doc_id)
    sim_docs_set = set()
    for couple in sim_docs:
        sim_docs_set.add(couple[0])
        sim_docs_set.add(couple[1])
        
    return sim_docs_set

# add a new column to the dataset containing the set of similar documents
dataset['similar_docs'] = dataset['doc_id'].apply(get_matching_doc_set)

dataset.head()

,Title,Short Description,Location,Price (Euro),Timestamp,Url Adv,doc_id,similar_docs
0,Studio accessoriato vicino metro A Furio Camillo,Affitto studio a professionisti preferibilment...,Roma,450,"12 ottobre, 11:32",https://www.kijiji.it/annunci/affitto/roma-ann...,0,"{0, 2561, 1153, 647, 1417, 2187, 779, 2319, 10..."
1,"Negozio 169Mq per laboratorio, ufficio, studio...","Privato affitta negozio 169 mq, al piano terra...",Prenestino / Casilino,1.700,"12 ottobre, 08:45",https://www.kijiji.it/annunci/affitto/roma-ann...,1,"{1, 1154, 2562, 648, 1418, 266, 780, 2188, 232..."
2,Negozio in tiburtina centro,Negozio c/1 roma tiburtina centro via eugenio ...,Tiburtino / Collatino,6.000,"17 October, 21:20",https://www.kijiji.it/annunci/affitto/roma-ann...,2,{}
3,Studio medico via anapo parco nemorense,"Studio medico avviato, composto da tre studi c...",Trieste / Somalia / Salario,200,"17 October, 20:22",https://www.kijiji.it/annunci/affitto/roma-ann...,3,{}
4,Cerco: Appartamento per donna lavoratrice refe...,"Donna lavoratrice, non residente, con reddito ...",Roma,Contatta l'utente,"17 October, 19:39",https://www.kijiji.it/annunci/affitto/roma-ann...,4,{}


In [31]:
new_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2822 entries, 0 to 2609
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Title              2822 non-null   object
 1   Short Description  2822 non-null   object
 2   Location           2822 non-null   object
 3   Price (Euro)       2822 non-null   object
 4   Timestamp          2822 non-null   object
 5   Url Adv            2822 non-null   object
 6   doc_id             2822 non-null   object
dtypes: object(7)
memory usage: 176.4+ KB
